In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import CuDNNLSTM


from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics


Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("./input/train.csv")

print("Train shape : ",train_df.shape)


Train shape :  (1306122, 3)


In [3]:
## Config values 
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2019)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=2020)

## fill up the missing values

train_X = train_df['question_text'].tolist()
train_X = [' '.join(t.split()[0:50]) for t in train_X]
train_X = np.array(train_X, dtype=object)[:, np.newaxis]

val_X = val_df['question_text'].tolist()
val_X = [' '.join(t.split()[0:50]) for t in val_X]
val_X = np.array(val_X, dtype=object)[:, np.newaxis]

test_X = test_df['question_text'].tolist()
test_X = [' '.join(t.split()[0:50]) for t in test_X]
test_X = np.array(test_X, dtype=object)[:, np.newaxis]


## Get the target values
train_y = train_df['target'].values
#train_y = np.array(train_y, dtype=object)[:, np.newaxis]
val_y = val_df['target'].values#
#val_y = np.array(val_y, dtype=object)[:, np.newaxis]
test_y = test_df['target'].values
#test_y = np.array(test_y, dtype=object)[:, np.newaxis]

print("Train shape :",train_X.shape, train_y.shape)
print("Validation shape : ", val_X.shape, val_y.shape)
print("Test shape : ",test_X.shape)

Train shape : (1044897, 1) (1044897,)
Validation shape :  (130612, 1) (130612,)
Test shape :  (130613, 1)


In [4]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions =1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [5]:
def build_model(): # uses crf from keras_contrib
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  dense = Dense(512, activation='relu')(embedding)
  dense = Dropout(0.3)(dense)
  dense = Dense(256, activation='relu')(dense)  
  dense = Dropout(0.1)(dense)
  pred = Dense(1, activation='sigmoid')(dense)
 

  model = Model(inputs=[input_text], outputs=pred)
  model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
  model.summary()
  
  return model

In [6]:
# Build and fit
model = build_model()
model.fit(train_X, train_y, batch_size=128, epochs=5, validation_data=(val_X, val_y))
model.save('./output/model_elmo_2.h5')
#model=  load_model('./output/model_elmo.h5')

#model = load_model('./output/model_elmo.h5')
# summarize model.
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.


W1125 08:28:47.802809 140160417371968 deprecation.py:323] From /home/user1/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1125 08:28:48.405180 140160417371968 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W1125 08:28:48.491325 140160417371968 deprecation.py:506] From /home/user1/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total para

W1125 08:28:48.867053 140160417371968 deprecation.py:323] From /home/user1/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 1044897 samples, validate on 130612 samples
Epoch 1/5
1044897/1044897 [==============================] - 5476s 5ms/step - loss: 0.1218 - acc: 0.9527 - val_loss: 0.1138 - val_acc: 0.9545
Epoch 2/5
1044897/1044897 [==============================] - 5460s 5ms/step - loss: 0.1143 - acc: 0.9552 - val_loss: 0.1120 - val_acc: 0.9553
Epoch 3/5
1044897/1044897 [==============================] - 5458s 5ms/step - loss: 0.1110 - acc: 0.9562 - val_loss: 0.1117 - val_acc: 0.9547
Epoch 4/5
1044897/1044897 [==============================] - 5465s 5ms/step - loss: 0.1087 - acc: 0.9571 - val_loss: 0.1090 - val_acc: 0.9561
Epoch 5/5
1044897/1044897 [==============================] - 5464s 5ms/step - loss: 0.1067 - acc: 0.9578 - val_loss: 0.1083 - val_acc: 0.9562


In [7]:
pred_elmo_val_y = model.predict([val_X], batch_size=128, verbose=1)
pred_elmo_val_y = np.squeeze(pred_elmo_val_y)
threshold = []
f1_array = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    threshold.append(thresh)
    f1_score = metrics.f1_score(val_y, (pred_elmo_val_y>thresh).astype(int))
    f1_array.append(f1_score)
    print("F1 score at threshold {0} is {1}".format(thresh, f1_score))

130612/130612 [==============================] - 601s 5ms/step
F1 score at threshold 0.1 is 0.5549848237216903
F1 score at threshold 0.11 is 0.5645438898450947
F1 score at threshold 0.12 is 0.5712764650477088
F1 score at threshold 0.13 is 0.5784625690839055
F1 score at threshold 0.14 is 0.5857014657493269
F1 score at threshold 0.15 is 0.593116084282162
F1 score at threshold 0.16 is 0.5988348823764842
F1 score at threshold 0.17 is 0.6046553672316384
F1 score at threshold 0.18 is 0.6079720472621948
F1 score at threshold 0.19 is 0.6125713216724348
F1 score at threshold 0.2 is 0.617094017094017
F1 score at threshold 0.21 is 0.6210318416108676
F1 score at threshold 0.22 is 0.6253909304143862
F1 score at threshold 0.23 is 0.6298720618863434
F1 score at threshold 0.24 is 0.6335459888481438
F1 score at threshold 0.25 is 0.6360909137133269
F1 score at threshold 0.26 is 0.6376841454620378
F1 score at threshold 0.27 is 0.6387685885729194
F1 score at threshold 0.28 is 0.6402748414376321
F1 score a

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure()
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.plot(threshold, f1_array)
plt.show()
fig.savefig('F1_score')

In [ ]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)
test_y = test_df['target'].values
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    f1_score = metrics.f1_score(test_y, (pred_glove_test_y>thresh).astype(int))
    print("F1 score at threshold {0} is {1}".format(thresh, f1_score))
